In [0]:
dbutils.widgets.text("table", "table") 
table =  dbutils.widgets.get("table")

# table = "vw_D21_3S_TRANS_OIL_AZURE_WEEK"


In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructField, StructType, StringType, ArrayType, LongType, TimestampType, BinaryType, IntegerType, DateType
from datetime import datetime
import time

file_info_schema = StructType([
    StructField('DATE', StringType(), True),
    StructField('STATION', StringType(), True),
    StructField('G', StringType(), True),
    StructField('QTY', StringType(), True),
    StructField('ETL_DATE', StringType(), True)
  
])

convert_udf = udf(lambda x: str(time.strftime("%Y-%m-%dT06:%M:%SZ", time.strptime(str(x),"%Y%m%d"))),StringType())  # python udf

raw_df = (spark.read.format('csv')
      .option('header',True)
      .schema(file_info_schema)
      .load(f'/mnt/source/{table}.csv')
      .withColumn('timestamp',convert_udf(F.col('DATE')))
      .withColumnRenamed('QTY','value')
      .select('timestamp','STATION','G','value')
      .repartition('station')
      .write
      .format('delta')
      .mode('overwrite')
      .save(f'/mnt/deltalake/{table}')
         )